<a href="https://colab.research.google.com/github/DS-Amarachi/Julia/blob/main/SpeedUp_ModExpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODULAR EXPONENTIATION ALGORITHMS IN CRYPTOGRAPHIC SYSTEMS**

##Python Implementations
Here we are using Python built-in tool and the custom tool(the classic repeated squaring method

3^3 mod 5 = 3X3X3 mod 5 = 27 mod 5 = 2

3^13 mod 7
13 = 1101
1X2^3 +1X2^2 + 0X2^1 + 1X2^0= 8 + 4 + 0 + 1
3^1 mod 7 = 3
3^0 mod 7 = 1
3^4 mod 7 = 81 mod 7 = 4
3^8 mod 7 = 2
3X1X4X2 = 24 mod 7 = 3

In [ ]:
import time

# Built-in Python modular exponentiation
def builtin_mod_exp(base, exp, mod):
    return pow(base, exp, mod)

# Custom modular exponentiation using repeated squaring
def mod_exp(base, exp, mod):
    result = 1
    base = base % mod  # Reduce base modulo mod initially

    while exp > 0:
        if exp % 2 == 1:  # If exponent is odd, multiply base with result
            result = (result * base) % mod
        exp = exp // 2  # Divide exponent by 2
        base = (base * base) % mod  # Square the base

    return result

# Benchmarking
def benchmark():
    base, exp, mod = 987654321, 123456, 1000000007

    start = time.time()
    result_builtin = builtin_mod_exp(base, exp, mod)
    end = time.time()
    print(f"Builtin pow() result: {result_builtin}, Time: {end - start:.6f} sec")

    start = time.time()
    result_custom = mod_exp(base, exp, mod)
    end = time.time()
    print(f"Custom mod_exp result: {result_custom}, Time: {end - start:.6f} sec")

benchmark()


Builtin pow() result: 443640203, Time: 0.000005 sec
Custom mod_exp result: 443640203, Time: 0.000008 sec


## First set up Julia

After the python implementation, install Julia and its packages

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.11.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.11.2 on the current Colab Runtime...
2025-02-20 20:39:39 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.11/julia-1.11.2-linux-x86_64.tar.gz [285843560/285843560] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.11

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Change the Runtime Type

Ensure to change the runtime type to Julia and GPU, save before running

In [ ]:
# Define the modular exponentiation function in Julia
function mod_exp(base::Int, exp::Int, mod::Int)
    result = 1
    base = base % mod  # Reduce base modulo mod initially

    while exp > 0
        if exp % 2 == 1  # If exponent is odd, multiply base with result
            result = (result * base) % mod
        end
        exp = exp ÷ 2  # Integer division (equivalent to exp // 2 in Python)
        base = (base * base) % mod  # Square the base
    end

    return result
end


mod_exp (generic function with 1 method)

In [ ]:
# Compute the execution time using the BenchmarkTools
using BenchmarkTools

function benchmark()
    base, exp, mod = 987654321, 123456, 1000000007

    @btime mod_exp($base, $exp, $mod)  # Benchmark custom function
    @btime powermod($base, $exp, $mod)  # Built-in optimized modular exponentiation
end

benchmark()


  263.680 ns (0 allocations: 0 bytes)
  404.145 ns (0 allocations: 0 bytes)


443640203

# Further Optimization

In cases of multiple bases using a multi-thread reduces execution time

In [ ]:
using BenchmarkTools, Base.Threads

function parallel_mod_exp(bases::Vector{Int}, exp::Int, mod::Int)
    results = Vector{Int}(undef, length(bases))

    @threads for i in eachindex(bases)
        results[i] = mod_exp(bases[i], exp, mod)
    end

    return results
end

# Example usage
bases = [123, 456, 789, 987654321]  # Multiple bases
mod_results = parallel_mod_exp(bases, 123456, 1000000007)
println(mod_results)

# Benchmark single-threaded execution
@time mod_results

[849990013, 144170895, 361310820, 443640203]
  0.000001 seconds


4-element Vector{Int64}:
 849990013
 144170895
 361310820
 443640203